# GEOG696C Spatiotemporal Data Analysis
## Homework #4
Alex Saunders | Last updated: 12 October 2023

In [6]:
from pathlib import Path
import os
import sys
import pandas as pd
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt, colors
from matplotlib.collections import LineCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.patheffects as pe
from datetime import datetime
import calendar
import scipy
import xarray
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature

In [2]:
# Set the root path
# rootPath = Path('C:/Users/alexa/Documents/GitHub/spatiotemporal_data_analysis/00_hw')
# rootPath = Path('C:/Users/alexsaunders/Documents/01_uoa/04_git/spatiotemporal_data_analysis/00_hw')
# rootPath = Path('C:/Users/alexsaunders/Documents/01_uoa/04_git/spatiotemporal_data_analysis/00_hw')
# dataPath = Path ('C:/Users/alexsaunders/Documents/01_uoa/01_study/2023/geog696c/data')
rootPath = Path('C:/Users/alexa/Documents/GitHub/spatiotemporal_data_analysis/00_hw')
dataPath = Path ('C:/Users/alexa/Documents/01_personal/07_phd/05_study/02_courses/2023/1/GEOG696/data')

## 3. Calculate and map the eigen modes of variability of winter Pacific SSTs
For the same domain as Deser and Blackon (1995)

### Initial processing of the SST data as prep for SVD

In [ ]:
pip uninstall netCDF4

In [7]:
# Load the SST data, HadISST Rayner et al 2003
sstData = xarray.open_dataset(dataPath/'HadISST_sst.nc') 
sst = sstData.sst

# Set missing values to na
sst = sst.where(sst>=0, np.nan)

# Transform the coord system from -180to180 longitude, to 0to360 degrees
sst = sst.assign_coords(longitude=(sst.longitude % 360))
# have to also sort the coordinates and associated data so they are in the correct order from 0 to 360
sst = sst.sortby(sst.longitude)

# Specify the desired area, and find corresponding start and end lat/lons that are present in the data
lons=[120, 260]
lats=[60, -20]
sst = sst.sel(longitude=slice(lons[0], lons[1]), latitude=slice(lats[0], lats[1]))

# Extract coordinates
lat = sst.latitude
lon = sst.longitude

# We use the rolling mean technique, with window of 5 months, taking only the ones ending in March
sstRolling = sst.rolling(time = 5).mean(skipna=True)

# Select March to get the average over NDJFM
endMo=3
sstSeasonal = sstRolling[sstRolling.time.dt.month==endMo]

# Drop the first which is an incomplete year
sstSeasonal = sstSeasonal.drop_isel(time=[0])

# Convert to np array for reshaping
ntime, nlat, nlon = sstSeasonal.shape
print(sstSeasonal.shape)
sstMat = np.array(sstSeasonal).reshape(ntime, nlat*nlon, order='F')
print(sstMat.shape)

# Drop locations wtih missing values
sstMatComplete = sstMat[:, ~np.isnan(sstMat).any(axis=0)]
print(sstMatComplete.shape)

# Get the mean and subtract it from the yearly seasonal average values, at ech location
sstAnom = sstMatComplete - sstMatComplete.mean(axis=0)


# Quick histogram to check the distribution of anomaly values centrs about zero
plt.hist(np.mean(sstAnom, axis=0))

ImportError: DLL load failed while importing _netCDF4: The specified procedure could not be found.

### Peform the SVD and get the EOFs (spatial) and PCs (temporal)
Here we will perform SVD on the data anomalies themselves, rather than the covariance matrix.

In [ ]:
# Retrieve the years, and the location lat lons (excluding the missing value locations)
years = range(sstSeasonal.time.dt.year.min().values, sstSeasonal.time.dt.year.max().values+1)
sstMatOrig=np.array(sstSeasonal).reshape(len(sstSeasonal.time), len(sstSeasonal.latitude)*len(sstSeasonal.longitude), order='F')
missingLocs = ~np.isnan(sstMatOrig).any(axis=0)
locLats=[]
locLons=[]
allLats=[]
allLons=[]
i=0
for lat in sstSeasonal.latitude.values:
    for lon in sstSeasonal.longitude.values:
        if missingLocs[i]==True:
            locLats.append(lat)
            locLons.append(lon)
        allLats.append(lat)
        allLons.append(lon)
        i=i+1        
latitudes = sst.latitude
longitudes = sst.longitude

In [ ]:
# Perform SVD
U, s, Vt = np.linalg.svd(sstAnom, full_matrices=False)
print(U.shape, s.shape, Vt.shape)
V=Vt.T

# Create the eofs, with dimensions of the original lat lon grid
eofs = np.zeros([nlat*nlon, ntime]) * np.nan # 2D
eofs[missingLocs, :] = V
eofs = eofs.reshape([nlat, nlon, ntime], order='F')
print(eofs.shape)

# Calculate the pcs
pcs = sstAnom @ V
print(pcs.shape)

# Get the % of variance explained by the eigen modes from the real data
eigenVals=(s**2)/(len(s)-1)
expVar = (s / np.sum(s)) * 100

### Map the EOFs

In [ ]:
levels = np.linspace(-0.05, 0.05, 51)

fig, axes = plt.subplots(2, 1, figsize=(12,12), constrained_layout=True, subplot_kw={'projection': ccrs.Miller(central_longitude=260)})

fig.suptitle('Map of the empirical orthgonal function (EOF) signals for {0}-{1}'.format(min(years), max(years)), fontsize=18)

# EOF1
ax=axes[0]
ax.set_title('EOF1', fontsize=16)
ax.coastlines(color="black") 
ax.add_feature(cfeature.LAND, color='lightgray')
ax.set_extent([lons[0], lons[1], lats[1], lats[0]], crs=ccrs.PlateCarree())
gl1 = ax.gridlines(color='k', linestyle='--', draw_labels=["left", "bottom"], x_inline=False,
                  y_inline=False, ylocs=[-10, 0, 10, 20, 30, 40, 50, 60], xlocs=[120, 140, 160, 180, -160, -140, -120, -100])
eof1Plot = ax.contourf(longitudes, latitudes, eofs[:,:,1], cmap='bwr', levels=levels,vmin=-0.05, vmax=0.05, transform=ccrs.PlateCarree()) 
cb1 = plt.colorbar(eof1Plot, orientation='vertical', ticks=np.linspace(-0.05, 0.05, 11))

# EOF2
ax=axes[1]
ax.set_title('EOF2', fontsize=16)
ax.coastlines(color="black") 
ax.add_feature(cfeature.LAND, color='lightgray')
ax.set_extent([lons[0], lons[1], lats[1], lats[0]], crs=ccrs.PlateCarree())
gl1 = ax.gridlines(color='k', linestyle='--', draw_labels=["left", "bottom"], x_inline=False,
                  y_inline=False, ylocs=[-10, 0, 10, 20, 30, 40, 50, 60], xlocs=[120, 140, 160, 180, -160, -140, -120, -100])
eof2Plot = ax.contourf(longitudes, latitudes, eofs[:,:,0], cmap='bwr', levels=levels,vmin=-0.05, vmax=0.05, transform=ccrs.PlateCarree()) 
cb1 = plt.colorbar(eof2Plot, orientation='vertical', ticks=np.linspace(-0.05, 0.05, 11))

ax.set_aspect('equal')

plt.show()

### Plot the PC time series

In [ ]:
fig, ax = plt.subplots(figsize=(12,5))

ax.set_title('Time series of the empirical orthgonal function (EOF) signals', fontsize=18)

ax.plot(years, pcs[:,0], label='EOF1')
ax.plot(years, pcs[:,1], label='EOF2')

ax.tick_params(which='both', labelsize=14)
ax.set_xlabel('Year', fontsize=14)
ax.set_ylabel('EOF Signal', fontsize=14)

ax.legend(fontsize=16, ncol=2)


plt.show()

## 4. Perform a Rule N significance test using Gaussian random noise conditioned on the data 

In [ ]:
sstAnom

In [ ]:
# Get mean and stdev of data values for conditioning the noise
mu = sstMatComplete.mean(axis=0)
sigma = sstMatComplete.std(axis=0)

In [ ]:
# Create empty list for saving eignevalues results
eigenValsAll=[]
for i in range(0, 100):
    # Create random Gaussian matrix conditioned on the data (with mean and stdev), then subtract the mean
    randomMat = np.random.normal(mu, sigma, sstMatComplete.shape) # sstAnom
    randomMat = (randomMat - randomMat.mean(axis=0)) / randomMat.std(axis=0)
    
    # Get eigen vectors and eigen values
    U, s, Vt = np.linalg.svd(randomMat, full_matrices=False)
    V=Vt.T
    eigenVals=(s**2)/(len(s)-1)
    # Save the eigen values
    eigenValsAll.append(eigenVals)
    
    # # calculate the covariance and then use svd to get the eigenvalues and eigenvectors
    # covMat = np.cov(randomMat, rowvar=False, ddof=1) # get the covariance of the random matrix
    # U, s, Vt = scipy.sparse.linalg.svds(covMat, k=153)
    # # U, s, Vt = np.linalg.svd(covMat) # factor the covariance matrix into eigenvalues and eigenvectors
    # eigenValsAll.append(s)

In [ ]:
# Get the 95%ile of the random data eigenvalues, for each mode
pct=0.95
modePcts=[]
for mode in range(eigenValsAll[0].shape[0]):
    eigenVals = [eigenVal[mode] for eigenVal in eigenValsAll]
    modePct = np.percentile(eigenVals, pct)
    modePcts.append(modePct)
    
# Convert the 95%ile eigenvalues for each mode into explained variance
expVarRandom = (modePcts / np.sum(modePcts)) * 100

# Determine which modes from our analysis 'pass' this significance test
modesPassed = np.array(range(1, len(expVar)+1))[expVar>expVarRandom]
print(modesPassed)

## 5. Perform a Rule N significance test using red autocorrelated noise

In [ ]:
# Determine a mean phi value to use based on first order autocorrelation in the data
acf1 = pd.DataFrame(sstAnom).apply(lambda x: x.autocorr())

# Create empty list for saving simulated red noise eigenvalues
eigenValsAll=[]

# loop through the j iterations with an inner loop through the i rows of the simulated red noise matrix
for j in range(0, 100):
    
    # Create an empty matrix to accept the red noise simulation
    redMat = np.full(sstAnom.shape, np.nan) 
    
    # mean value and the variance of the random distribution
    c = sstAnom.mean(axis=0) # we have a zero-mean process
    sigma_e = np.sqrt((1 - acf1 ** 2))  # sets the appropriate standard deviation for [0,1] data

    # set the first value in the array as a random draw with the correct mean and variance
    redMat[0,:] = np.array([c + np.random.normal(0, sigma_e, size=sstAnom.shape[1])])

    # inner loop i that goes row-by-row to generate the red noise simulation
    for i in range(1, sstAnom.shape[0]):
        redMat[i,:] = np.array([c + (acf1 * redMat[i-1,:]) + np.random.normal(0, sigma_e, size=sstAnom.shape[1])])
    
    # Get eigen vectors and eigen values
    U, s, Vt = np.linalg.svd(redMat, full_matrices=False)
    V=Vt.T
    eigenVals=(s**2)/(len(s)-1)
    # Save the eigen values
    eigenValsAll.append(eigenVals)
    
    # # calculate the covariance and then use svd to get the eigenvalues and eigenvectors
    # covMat = np.cov(redMat, rowvar=False, ddof=1) # get the covariance of the random matrix
    # U, s, Vt = np.linalg.svd(covMat, k=153) # factor the covariance matrix into eigenvalues and eigenvectors
    # eigenValsAll.append(s)

In [ ]:
# Get the 95%ile of the random data eigenvalues, for each mode
pct=0.95
modePcts=[]
for mode in range(eigenValsAll[0].shape[0]):
    eigenVals = [eigenVal[mode] for eigenVal in eigenValsAll]
    modePct = np.percentile(eigenVals, pct)
    modePcts.append(modePct)
    
# Convert the 95%ile eigenvalues for each mode into explained variance
expVarRed = (modePcts / np.sum(modePcts)) * 100

# Determine which modes from our analysis 'pass' this significance test
modesPassed = np.array(range(1, len(expVar)+1))[expVar>expVarRed]
print(modesPassed)

### Plot the Rule N test with a scree plot

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

markerline, stemlines, baseline = ax.stem(range(1, len(expVar) + 1), expVar, label='Real data')#, use_line_collection=True)

plt.setp(stemlines, color='k')
plt.setp(markerline, marker='o', markersize=10, color='k', zorder=2)
plt.setp(baseline, color='none', zorder=1)

ax.plot(range(1, len(expVarRandom)+1), expVarRandom, color='blue', linestyle='--', label='Rule N Gaussian Noise')
ax.plot(range(1, len(expVarRed)+1), expVarRed, color='red', linestyle='--', label='Rule N Red Noise')


ax.set_xlabel('Eigen Rank', fontsize=14)
ax.set_ylabel('% of total variance explained', fontsize=14)
ax.set_xticks(range(1, len(expVar) + 1))
ax.set_xlim(0, 30)
ax.set_ylim(bottom=0)

ax.legend()

plt.show()

__How many modes of North Pacific winter SSTs are "significant, meaningful, interpretable and/or differentiated from noise?"__

## 6. Perform a 'field correlation' analysis
The correlation between a single time series and the time series at every point in the field

In [ ]:
# Get data and PCs in xarray for easy correlation
pcsXr = xarray.DataArray(pcs,
                        dims=("time","mode"),
                        coords=[
                            ("time", sstSeasonal.time.values),
                            ("mode", range(1,len(years)+1)),
                        ],)

# sstAnomXr = xarray.DataArray(sstAnom,
#                         dims=("year","loc"),
#                         coords=[
#                             ("year", years),
#                             ("loc", range(1,sstAnom.shape[1]+1)),
#                         ],)

# For EOF 1 and EOF 2 time series, compute correlation with the original SST data
fieldCorr = xarray.corr(sstSeasonal, pcsXr, dim="time")

### Plot maps of field correlation for the first two modes

In [ ]:
levels = np.linspace(-1, 1, 21)

fig, axes = plt.subplots(2, 1, figsize=(12,12), constrained_layout=True, subplot_kw={'projection': ccrs.Miller(central_longitude=260)})

fig.suptitle('Map of the EOF field correlation for {0}-{1}'.format(min(years), max(years)), fontsize=18)

# EOF1
ax=axes[0]
ax.set_title('EOF1', fontsize=16)
ax.coastlines(color="black") 
ax.add_feature(cfeature.LAND, color='lightgray')
ax.set_extent([lons[0], lons[1], lats[1], lats[0]], crs=ccrs.PlateCarree())
gl1 = ax.gridlines(color='k', linestyle='--', draw_labels=["left", "bottom"], x_inline=False,
                  y_inline=False, ylocs=[-10, 0, 10, 20, 30, 40, 50, 60], xlocs=[120, 140, 160, 180, -160, -140, -120, -100])
eof1Plot = ax.contourf(longitudes, latitudes, fieldCorr.sel(mode=1).values, cmap='BrBG', levels=levels, vmin=-1, vmax=1, transform=ccrs.PlateCarree()) 
cb1 = plt.colorbar(eof1Plot, orientation='vertical', ticks=np.linspace(-1, 1, 11))

# EOF2
ax=axes[1]
ax.set_title('EOF2', fontsize=16)
ax.coastlines(color="black") 
ax.add_feature(cfeature.LAND, color='lightgray')
ax.set_extent([lons[0], lons[1], lats[1], lats[0]], crs=ccrs.PlateCarree())
gl1 = ax.gridlines(color='k', linestyle='--', draw_labels=["left", "bottom"], x_inline=False,
                  y_inline=False, ylocs=[-10, 0, 10, 20, 30, 40, 50, 60], xlocs=[120, 140, 160, 180, -160, -140, -120, -100])
eof2Plot = ax.contourf(longitudes, latitudes, fieldCorr.sel(mode=2).values, cmap='BrBG', levels=levels, vmin=-1, vmax=1, transform=ccrs.PlateCarree()) 
cb1 = plt.colorbar(eof2Plot, orientation='vertical', ticks=np.linspace(-1, 1, 11))

ax.set_aspect('equal')

plt.show()

__How do the maps compare with the results from Deser and Blackmon (1995)?__

## 7 Identify the years with the highest and lowest of the signal values, for EOF 1

In [ ]:
upper=pcsXr.sel(mode=1).quantile(0.90)
lower=pcsXr.sel(mode=1).quantile(0.10)

upperYears= [pd.Timestamp(item).year for item in pcsXr.sel(mode=1).time[pcsXr.sel(mode=1) >= upper].values]
lowerYears= [pd.Timestamp(item).year for item in pcsXr.sel(mode=1).time[pcsXr.sel(mode=1) <= lower].values]

print('Upper 10%ile: ', upperYears)
print('Lower 10%ile: ', lowerYears)

## 8. Import the reanalysis 500mb height data and slice for the same time period as the SST

In [ ]:
(dataPath/'hgt.500mb.mon.mean.nc').exists()

In [ ]:
# Load the SST data, HadISST Rayner et al 2003
hgtData = xarray.open_dataset(dataPath/'hgt_500mb_mon_mean.nc') 
# hgt = sstData.sst